## 点云特征提取

### 法线估计

In [4]:
import open3d as o3d 
import numpy as np 
print("->正在加载点云... ") 
pcd = o3d.io.read_point_cloud("test1.pcd") 
print(pcd) 
print("->正在估计法线并可视化...") 
radius = 0.01 # 搜索半径 
max_nn = 30 # 邻域内用于估算法线的最大点数 
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius, max_nn)) # 执行法线估计 
o3d.visualization.draw_geometries([pcd], point_show_normal=True) 
print("->正在打印前10个点的法向量...") 
print(np.asarray(pcd.normals)[:10, :])


->正在加载点云... 
PointCloud with 801147 points.
->正在估计法线并可视化...
->正在打印前10个点的法向量...
[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [7]:
pcd = o3d.io.read_point_cloud("beam_1.txt", format = 'xyzn')
o3d.visualization.draw_geometries([pcd], point_show_normal=True) 

## 传统点云分割
### DBSCAN 聚类分割

In [10]:
import open3d as o3d 
import numpy as np 
import matplotlib.pyplot as plt 
print("->正在加载点云... ") 
pcd = o3d.io.read_point_cloud("test.pcd") 
print(pcd) 
print("->正在DBSCAN聚类...") 
eps = 0.5 # 同一聚类中最大点间距 
min_points = 50 # 有效聚类的最小点数 
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm: 
    labels = np.array(pcd.cluster_dbscan(eps, min_points, print_progress=True)) 
    max_label = labels.max() # 获取聚类标签的最大值 [-1,0,1,2,...,max_label]，label = -1 为噪声，因此总聚类个数为 max_label + 1 
    print(f"point cloud has {max_label + 1} clusters") 
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1)) 
    colors[labels < 0] = 0 # labels = -1 的簇为噪声，以黑色显示 
    pcd.colors = o3d.utility.Vector3dVector(colors[:, :3]) 
o3d.visualization.draw_geometries([pcd])


其它平面分割，隐藏点剔除，暂时略过

点云曲面重建也先略过

## 点云空间变换

### Translate

```
pcd.translate((tx,ty,tz),relative=True)
```
relative 默认为 True, 实现平移

relative 设置为 False, 则将点云移动到满足质心为 (tx, ty, tz) 的位置


In [10]:
import copy # 点云深拷贝 
import open3d as o3d 
# -------------------------- 加载点云 ------------------------ 
print("->正在加载点云... ") 
pcd = o3d.io.read_point_cloud("test1.pcd") 
print(pcd) 
print(f'pcd质心：{pcd.get_center()}') 
# =========================================================== 
# # -------------------------- 点云平移 ------------------------ 
print("\n->沿X轴平移1200m") 
pcd_tx = copy.deepcopy(pcd).translate((1200, 0, 0)) 
pcd_tx.paint_uniform_color([1, 0, 0]) 
print(pcd_tx) 
print(f'pcd_tx质心：{pcd_tx.get_center()}') 
print("\n->沿Y轴平移1400m") 
pcd_ty = copy.deepcopy(pcd_tx).translate((0, 1400, 0)) 
pcd_ty.paint_uniform_color([0, 1, 0]) 
print(pcd_ty) 
print(f'pcd_ty质心：{pcd_ty.get_center()}') 
print("\n->沿X轴平移-2000m，再沿Y轴平移2000m") 
pcd_txy = copy.deepcopy(pcd).translate((-2000, 2000, 0)) 
pcd_txy.paint_uniform_color([0, 0, 1]) 
print(pcd_txy) 
print('pcd_txy质心：', pcd_txy.get_center()) 
# =========================================================== 
# -------------------------- 可视化 -------------------------- 
o3d.visualization.draw_geometries([pcd, pcd_tx, pcd_ty, pcd_txy]) 
# ===========================================================


->正在加载点云... 
PointCloud with 801147 points.
pcd质心：[605.02394567 616.62997802  54.76084913]

->沿X轴平移1200m
PointCloud with 801147 points.
pcd_tx质心：[1805.02394567  616.62997802   54.76084913]

->沿Y轴平移1400m
PointCloud with 801147 points.
pcd_ty质心：[1805.02394567 2016.62997802   54.76084913]

->沿X轴平移-2000m，再沿Y轴平移2000m
PointCloud with 801147 points.
pcd_txy质心： [-1394.97605433  2616.62997802    54.76084913]


In [13]:
import copy # 点云深拷贝 
import open3d as o3d 
# -------------------------- 加载点云 ------------------------ 
print("->正在加载点云... ") 
pcd = o3d.io.read_point_cloud("test.pcd") 
print(pcd) 
print(f'pcd质心：{pcd.get_center()}') 
# =========================================================== 
# # -------------------------- 点云平移 ------------------------ 
print("\n->沿X轴平移0.2m")
pcd_tx = copy.deepcopy(pcd).translate((0.4, 0.4, 0.4)) 
pcd_tx.paint_uniform_color([1, 0, 0]) 
print(pcd_tx) 
print(f'pcd_tx质心：{pcd_tx.get_center()}') 
print("\n->将点云质心平移到指定位置") 
pcd_new = copy.deepcopy(pcd_tx).translate((0.4, 0.4, 0.4),relative=False) # pcd_new = copy.deepcopy(pcd_tx).translate((0.2, 0.2, 0.2),False) # relative 可以省略 
pcd_new.paint_uniform_color([0, 1, 0]) 
print(pcd_new) 
print(f'pcd_new：{pcd_new.get_center()}') 
# =========================================================== 
# # -------------------------- 可视化 -------------------------- 
o3d.visualization.draw_geometries([pcd, pcd_tx, pcd_new]) 
# ===========================================================


->正在加载点云... 
PointCloud with 36980 points.
pcd质心：[ 0.00098508 -0.00621785  0.02644219]

->沿X轴平移0.2m
PointCloud with 36980 points.
pcd_tx质心：[0.40098508 0.39378215 0.42644219]

->将点云质心平移到指定位置
PointCloud with 36980 points.
pcd_new：[0.4 0.4 0.4]


### 旋转
Rotation
rotate 需要两个参数，旋转矩阵R和旋转中心
如果不指定旋转中心，则围绕点云质心旋转，旋转前后质心位置不变。

旋转矩阵可以通过多种方式获得。欧拉角(xyz,zxy,xzy,yxz)，轴角，四参数等

In [17]:
import copy # 点云深拷贝 
import open3d as o3d 
import numpy as np 
# -------------------------- 加载点云 ------------------------ 
print("->正在加载点云... ") 
pcd = o3d.io.read_point_cloud("test.pcd") 
print(pcd) 
pcd.paint_uniform_color([1,0,0]) 
print("->pcd质心：",pcd.get_center()) 
# =========================================================== 
# # -------------------------- 点云旋转 ------------------------ 
print("\n->采用欧拉角进行点云旋转") 
pcd_EulerAngle = copy.deepcopy(pcd) 
R1 = pcd.get_rotation_matrix_from_xyz((0,np.pi/2,0)) 
print("旋转矩阵：\n",R1) 
pcd_EulerAngle.rotate(R1) # 不指定旋转中心 
pcd_EulerAngle.paint_uniform_color([0,0,1]) 
print("\n->pcd_EulerAngle质心：",pcd_EulerAngle.get_center()) 
# =========================================================== 
# # -------------------------- 可视化 -------------------------- 
o3d.visualization.draw_geometries([pcd, pcd_EulerAngle]) # ===========================================================

->正在加载点云... 
PointCloud with 36980 points.
->pcd质心： [ 0.00098508 -0.00621785  0.02644219]

->采用欧拉角进行点云旋转
旋转矩阵：
 [[ 6.123234e-17  0.000000e+00  1.000000e+00]
 [ 0.000000e+00  1.000000e+00  0.000000e+00]
 [-1.000000e+00  0.000000e+00  6.123234e-17]]

->pcd_EulerAngle质心： [ 0.00098508 -0.00621785  0.02644219]


In [19]:
import copy # 点云深拷贝 
import open3d as o3d 
import numpy as np 
# -------------------------- 加载点云 ------------------------ 
print("->正在加载点云... ") 
pcd = o3d.io.read_point_cloud("test.pcd") 
print(pcd) 
pcd.paint_uniform_color([1,0,0]) 
print("->pcd质心：",pcd.get_center()) 
# =========================================================== 
# -------------------------- 点云旋转 ------------------------ 
print("\n->采用欧拉角进行点云旋转") 
pcd_EulerAngle = copy.deepcopy(pcd) 
R1 = pcd.get_rotation_matrix_from_xyz((0,np.pi/2,0)) 
print("旋转矩阵：\n",R1) 
pcd_EulerAngle.rotate(R1,center = (0.1,0.1,0.1)) # 指定旋转中心 p
pcd_EulerAngle.paint_uniform_color([0,0,1]) 
print("\n->pcd_EulerAngle质心：",pcd_EulerAngle.get_center()) 
# =========================================================== 
#-------------------------- 可视化 --------------------------
o3d.visualization.draw_geometries([pcd, pcd_EulerAngle]) 
# ===========================================================


->正在加载点云... 
PointCloud with 36980 points.
->pcd质心： [ 0.00098508 -0.00621785  0.02644219]

->采用欧拉角进行点云旋转
旋转矩阵：
 [[ 6.123234e-17  0.000000e+00  1.000000e+00]
 [ 0.000000e+00  1.000000e+00  0.000000e+00]
 [-1.000000e+00  0.000000e+00  6.123234e-17]]

->pcd_EulerAngle质心： [ 0.02644219 -0.00621785  0.19901492]


## 缩放
也是使用Rotate 实现
缩放操作中必须指定缩放后点云质心


In [26]:
import copy # 点云深拷贝 
import open3d as o3d 
import numpy as np 
# -------------------------- 加载点云 ------------------------ 
print("->正在加载点云... ") 
pcd = o3d.io.read_point_cloud("test.pcd") 
print(pcd) 
pcd.paint_uniform_color([1,0,0]) 
print("->pcd质心：",pcd.get_center()) 
# =========================================================== 
# -------------------------- 点云缩放 ------------------------ 
print("\n->点云缩放") 
pcd_scale1 = copy.deepcopy(pcd) 
pcd_scale1.scale(20,center=pcd.get_center()) 
pcd_scale1.paint_uniform_color([0,0,1]) 
print("->pcd_scale1质心：",pcd_scale1.get_center()) # 缩放前后质心不变 
pcd_scale2 = copy.deepcopy(pcd) 
pcd_scale2.scale(0.5,center=(1,1,1)) # 自定义缩放后的质心 
pcd_scale2.paint_uniform_color([0,1,0]) 
print("->pcd_scale2质心：",pcd_scale2.get_center()) 
# =========================================================== 
# # -------------------------- 可视化 -------------------------- 
o3d.visualization.draw_geometries([pcd, pcd_scale1,pcd_scale2]) 
# ===========================================================


->正在加载点云... 
PointCloud with 36980 points.
->pcd质心： [ 0.00098508 -0.00621785  0.02644219]

->点云缩放
->pcd_scale1质心： [ 0.00098508 -0.00621785  0.02644219]
->pcd_scale2质心： [0.50049254 0.49689107 0.5132211 ]


In [29]:
import copy # 点云深拷贝 
import open3d as o3d 
import numpy as np 
# -------------------------- 加载点云 ------------------------ 
print("->正在加载点云... ") 
pcd = o3d.io.read_point_cloud("test1.pcd") 
print(pcd) 
pcd.paint_uniform_color([1,0,0]) 
print("->pcd质心：",pcd.get_center()) 
# =========================================================== 
# # -------------------------- transform ------------------------ 
print("\n->点云的一般变换") 
pcd_T = copy.deepcopy(pcd) 
T = np.eye(4) 
T[ :3, :3] = pcd.get_rotation_matrix_from_xyz((np.pi/6,np.pi/4,0)) # 旋转矩阵 
T[0,3] = 5.0 # 平移向量的dx 
T[1,3] = 3.0 # 平移向量的dy 
print("\n->变换矩阵：\n",T) 
pcd_T.transform(T) 
pcd_T.paint_uniform_color([0,0,1]) 
print("\n->pcd_scale1质心：",pcd_T.get_center()) 
# =========================================================== 
# -------------------------- 可视化 -------------------------- 
o3d.visualization.draw_geometries([pcd, pcd_T]) 
# ===========================================================


->正在加载点云... 
PointCloud with 801147 points.
->pcd质心： [605.02394567 616.62997802  54.76084913]

->点云的一般变换

->变换矩阵：
 [[ 0.70710678  0.          0.70710678  5.        ]
 [ 0.35355339  0.8660254  -0.35355339  3.        ]
 [-0.61237244  0.5         0.61237244  0.        ]
 [ 0.          0.          0.          1.        ]]

->pcd_scale1质心： [471.53830253 731.5646092  -28.65096369]
